In [ ]:
from optimpv import *
from optimpv.axBOtorch.axUtils import *
import pandas as pd
import numpy as np
from scipy import interpolate
import warnings

import numpy as np
import torch
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import init_notebook_plotting, render

# import logging

# from ray import train, tune
# from ray.tune.schedulers import AsyncHyperBandScheduler
# from ray.tune.search.ax import AxSearch


init_notebook_plotting()

# Suppress FutureWarning messages
# warnings.simplefilter(action='ignore', category=FutureWarning)
# ##part of the message is also okay
warnings.filterwarnings('ignore') 

mun = FitParam(name = 'l2.mu_n', value = 3e-5, bounds = [1e-5,1e-3], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\mu_n$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)

mup = FitParam(name = 'l2.mu_p', value = 8e-4, bounds = [1e-5,1e-3], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\mu_p$', unit='m$^2$ V$^{-1}$s$^{-1}$', axis_type = 'log', std = 0,encoding = None,force_log = False)

bulk_tr = FitParam(name = 'l2.N_t_bulk', value = 4e20, bounds = [1e19,1e21], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$N_{T}$', unit='s', axis_type = 'log', std = 0,encoding = None,force_log = False)

# W_L = FitParam(name = 'W_L', value = 4.05, bounds = [3.92,4.1], values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$W_L$', unit='eV', axis_type = 'linear', std = 0,encoding = None,force_log = False)

# C_np_bulk = FitParam(name = 'l2.C_np_bulk', value = 1e-13, bounds = [1e-14,1e-12], values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = True, stepsize = None, display_name=r'$C_{np}$', unit='m$^{-3}$', axis_type = 'log', std = 0,encoding = None,force_log = False)

offset_l2_l1 = FitParam(name = 'offset_l2_l1.E_c', value = -0.1, bounds = [-0.2,0.0], values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\Delta E_{L2-L1}$', unit='eV', axis_type = 'linear', std = 0,encoding = None,force_log = False)

# offset_l3_l2 = FitParam(name = 'offset_l3_l2.E_c', value = -1.65, bounds = [-2,0], values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\Delta E_{L3-L2}$', unit='eV', axis_type = 'linear', std = 0,encoding = None,force_log = False)

offset_l2_l3 = FitParam(name = 'offset_l2_l3.E_v', value = 0.1, bounds = [0,0.3], values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\Delta E_{L2-L3}$', unit='eV', axis_type = 'linear', std = 0,encoding = None,force_log = False)

Egap_l1 = FitParam(name = 'Egap_l1.E_v', value = 1.6, bounds = [1.55,1.65], type='fixed', values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$E_{gap,L1}$', unit='eV', axis_type = 'linear', std = 0,encoding = None,force_log = False)

offset_W_L = FitParam(name = 'offset_W_L.E_c', value = -0.1, bounds = [-0.2,0], type='fixed', values = None, start_value = None, log_scale = False, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$\Delta W_L$', unit='eV', axis_type = 'linear', std = 0,encoding = None,force_log = False)

Nions = FitParam(name = 'l2.N_ions', value = 1e22, bounds = [1e20,5e22], type='range', values = None, start_value = None, log_scale = True, value_type = 'float', fscale = None, rescale = False, stepsize = None, display_name=r'$C_{ions}$', unit='m$^{-3}$', axis_type = 'log', std = 0,encoding = None,force_log = False)


params = [mun, mup, bulk_tr, offset_l2_l1, offset_l2_l3, Egap_l1, offset_W_L, Nions]

# ({'l2.mu_n': 0.001, 'l2.mu_p': 1e-05, 'l2.N_t_bulk': 1.7603940470752775e+20, 'l2.C_np_bulk': 2.737249616717965, 'offset_l2_l1.E_c': -0.07161177367482424, 'offset_l2_l3.E_v': 0.18380439095587323, 'Egap_l1.E_v': 1.6, 'offset_W_L.E_c': -0.1}, ({'rmsre': np.float64(-0.22903756081756554)}, {'rmsre': {'rmsre': np.float64(0.0016075557479600649)}}))


In [ ]:
import pySIMsalabim as sim
from pySIMsalabim.experiments.JV_steady_state import *

session_path = os.path.join('/home/lecorre/Desktop/pySIMsalabim/', 'SIMsalabim','SimSS')
simss_device_parameters = os.path.join(session_path, 'simulation_setup.txt')

# Set the JV parameters
Gfracs = [0.1,0.5,1.0] # Fractions of the generation rate to simulate
# Gfracs = None
UUID = str(uuid.uuid4())

cmd_pars = []
for param in params:
    if param.name != 'l2.C_np_bulk' and param.name != 'offset_l2_l1.E_c' and param.name != 'offset_l2_l3.E_v' and param.name != 'Egap_l1.E_v' and param.name != 'offset_W_L.E_c' and param.name != 'l2.N_ions':
        cmd_pars.append({'par':param.name, 'val':str(param.value)})
    elif param.name == 'offset_l2_l1.E_c':
        cmd_pars.append({'par':'l1.E_c', 'val':str(3.9-param.value)})
        vv = 3.9-param.value
    elif param.name == 'l2.N_ions':
        cmd_pars.append({'par':'l2.N_cation', 'val':str(param.value)})
        cmd_pars.append({'par':'l2.N_anion', 'val':str(param.value)})
    elif param.name == 'l2.C_np_bulk':
        cmd_pars.append({'par':'l2.C_n_bulk', 'val':str(param.value)})
        cmd_pars.append({'par':'l2.C_p_bulk', 'val':str(param.value)})

    elif param.name == 'offset_l2_l3.E_v':
        cmd_pars.append({'par':'l3.E_v', 'val':str(5.53-param.value)})
    
    elif param.name == 'Egap_l1.E_v':
        cmd_pars.append({'par':'l1.E_v', 'val': str(vv+param.value)})
    
    elif param.name == 'offset_W_L.E_c':
        cmd_pars.append({'par':'W_L', 'val':str(vv-param.value)})


print(cmd_pars)

# Run the JV simulation
ret, mess = run_SS_JV(simss_device_parameters, session_path, JV_file_name = 'JV.dat', varFile= 'Var.dat',G_fracs = Gfracs, parallel = True, max_jobs = 3, UUID=UUID, cmd_pars=cmd_pars)


# save data for fitting
X,y = [],[]
if Gfracs is None:
    data = pd.read_csv(os.path.join(session_path, 'JV_'+UUID+'.dat'), sep=r'\s+') # Load the data
    Vext = np.asarray(data['Vext'].values)
    Jext = np.asarray(data['Jext'].values)
    G = np.ones_like(Vext)

    X= Vext
    y = Jext

    plt.figure()
    plt.plot(X,y)
    plt.show()
else:
    for Gfrac in Gfracs:
        data = pd.read_csv(os.path.join(session_path, 'JV_Gfrac_'+str(Gfrac)+'_'+UUID+'.dat'), sep=r'\s+') # Load the data
        Vext = np.asarray(data['Vext'].values)
        Jext = np.asarray(data['Jext'].values)
        G = np.ones_like(Vext)*Gfrac

        if len(X) == 0:
            X = np.vstack((Vext,G)).T
            y = Jext
        else:
            X = np.vstack((X,np.vstack((Vext,G)).T))
            y = np.hstack((y,Jext))

    print(X.shape)
    print(y.shape)


    plt.figure()
    for Gfrac in Gfracs:
        plt.plot(X[X[:,1]==Gfrac,0],y[X[:,1]==Gfrac],label='Gfrac = '+str(Gfrac))
    plt.legend()
    plt.show()




In [ ]:
from optimpv.DDfits.JVAgent import JVAgent
metric = 'rmsre'
# metric = 'mse'
loss = 'log10'
# loss = 'linear'

jv = JVAgent(params, X, y, session_path, simss_device_parameters,parallel = True, max_jobs = 3, metric = metric, loss = loss)

In [ ]:
from optimpv.axBOtorch.axBOtorchOptimizer import axBOtorchOptimizer
from botorch.acquisition.logei import (
    qLogExpectedImprovement,
    qLogNoisyExpectedImprovement,
)
from ax.modelbridge.transforms.standardize_y import StandardizeY
from ax.modelbridge.transforms.unit_x import UnitX
from ax.modelbridge.transforms.remove_fixed import RemoveFixed
from ax.modelbridge.transforms.log import Log
model_kwargs_list = [{'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double},{'torch_device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),'torch_dtype': torch.double,'botorch_acqf_class':qLogNoisyExpectedImprovement,'transforms':[RemoveFixed, Log,UnitX, StandardizeY]}]



optimizer = axBOtorchOptimizer(params = params, agents = jv, models = ['SOBOL','BOTORCH_MODULAR'],n_batches = [1,80], batch_size = [10,2], metrics = metric, minimize_list = True, thresholds = None, ax_client = None,  max_parallelism = 10,model_kwargs_list = None, model_gen_kwargs_list = None, name = 'ax_opti')


In [ ]:
optimizer.optimize()

In [ ]:
ax_client = optimizer.ax_client
best_parameters = ax_client.get_best_parameters()
print(best_parameters[0])
jv.params_w(best_parameters[0],jv.params)
print(jv.get_SIMsalabim_clean_cmd(jv.params))
jv.package_SIMsalabim_files(jv.params,'simss')

In [ ]:
# plot the evolution of the optimization
render(ax_client.get_contour_plot(param_x="l2.mu_n", param_y="l2.mu_p", metric_name=optimizer.all_metrics[0]))
# render(ax_client.get_contour_plot(param_x="l2.N_t_bulk", param_y="l2.C_np_bulk", metric_name=metric))
from ax.plot.slice import plot_slice
model = ax_client.generation_strategy.model

render(plot_slice(model=model, param_name="l2.mu_n", metric_name=optimizer.all_metrics[0]))
render(plot_slice(model=model, param_name="l2.mu_p", metric_name=optimizer.all_metrics[0]))
render(plot_slice(model=model, param_name="l2.N_t_bulk", metric_name=optimizer.all_metrics[0]))
# render(plot_slice(model=model, param_name="l2.C_np_bulk", metric_name=metric))



In [ ]:
data = ax_client.experiment.fetch_data()

plt.plot(np.minimum.accumulate(data.df["mean"]), label="Best value seen so far")

# plt.yscale("log")


In [ ]:
# create dic with keys same as ax_client.experiment.trials[0].arm.parameters
dumdic = {}
for key in ax_client.experiment.trials[0].arm.parameters.keys():
    dumdic[key] = []
from ax.core.base_trial import TrialStatus as T
# fill the dic with the values of the parameters
for i in range(len(ax_client.experiment.trials)):
    if ax_client.experiment.trials[i].status == T.COMPLETED:
        for key in ax_client.experiment.trials[i].arm.parameters.keys():
            dumdic[key].append(ax_client.experiment.trials[i].arm.parameters[key])

data = ax_client.experiment.fetch_data().df
# print(data)
mse = data['mean']
dumdic['mse'] = mse

df = pd.DataFrame(dumdic)
# df = df.sort_values(by='mse')


In [ ]:
import seaborn as sns
plt.figure()

xx = 'l2.N_t_bulk'
yy = 'offset_l2_l1.E_c'
kind = 'kde'
if kind == 'scatter':
    g = sns.jointplot(x=xx,y=yy,data=df,kind=kind, cmap="rocket",color="#03051A",space=0, marginal_ticks=False)
    plt.xscale('log')
    plt.yscale('log')
else:
    g = sns.jointplot(x=xx,y=yy,data=df,kind=kind,fill=True, thresh=0, levels = 100, cmap="rocket",color="#03051A",space=0, marginal_ticks=False,log_scale=(True,False))
# plt.scatter(3e-5,8e-4,c='r')
plt.scatter(best_parameters[0][xx],best_parameters[0][yy],c='y',marker='*',s=200)


plt.show()

In [ ]:
plt.figure()

xx = 'l2.mu_n'
yy = 'l2.mu_p'
kind = 'kde'
if kind == 'scatter':
    g = sns.jointplot(x=xx,y=yy,data=df,kind=kind, cmap="rocket",color="#03051A",space=0, marginal_ticks=False)
    plt.xscale('log')
    plt.yscale('log')
else:
    g = sns.jointplot(x=xx,y=yy,data=df,kind=kind,fill=True, thresh=0, levels = 100, cmap="rocket",color="#03051A",space=0, marginal_ticks=False,log_scale=(True,True))
# plt.scatter(3e-5,8e-4,c='r')
plt.scatter(best_parameters[0][xx],best_parameters[0][yy],c='y',marker='*',s=200)


plt.show()

In [ ]:
# rerun the simulation with the best parameters
yfit = jv.run(parameters=ax_client.get_best_parameters()[0])
# print(jv.run_Ax(parameters=results.get_best_result(metric=metric,mode='min',filter_nan_and_inf=True).config))
plt.figure(figsize=(20,20))
for Gfrac in Gfracs:
    plt.plot(X[X[:,1]==Gfrac,0],y[X[:,1]==Gfrac],label='Gfrac = '+str(Gfrac))
    plt.plot(X[X[:,1]==Gfrac,0],yfit[X[:,1]==Gfrac],label='Gfrac = '+str(Gfrac)+' fit',linestyle='--')
plt.legend()
plt.show()

In [ ]:
# Clean up the output files (comment out if you want to keep the output files)
sim.clean_all_output(session_path)
sim.delete_folders('tmp',session_path)